                  CHỦ ĐỀ:  ỨNG DỤNG MÔ HÌNH HỌC MÁY VÀO DỰ ĐOÁN SỐ ĐIỆN SỬ DỤNG TRONG THÁNG


Cách sử dụng trên Google Colab:

Bước 1: Cài đặt kagglehub (chạy cell đầu tiên)

    !pip install kagglehub[pandas-datasets]

Bước 2: Xác thực Kaggle (Ở đây dataset ta chọn electric-power-consumption-data-set là public nên không cần xác thực)

Bước 3: Chạy các cell theo thứ tự

In [10]:
# 1. Cài đặt kagglehub
!pip install kagglehub[pandas-datasets]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import kagglehub

# Tải dataset
path = kagglehub.dataset_download("uciml/electric-power-consumption-data-set")
print("Path to dataset files:", path)


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Path to dataset files: C:\Users\Admin\.cache\kagglehub\datasets\uciml\electric-power-consumption-data-set\versions\1
